<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/Cnyes_category_stock_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><span style="font-size:20px"><strong>鉅亨網新聞主題爬蟲（精確版，用於Colab平台）</strong></span></h1>

<p>可自訂主題新聞與日期區間爬取新聞內文，並可設定是否爬取頭條以外的新聞<br />
<span style="color:null"><strong>輸出資料的欄位依序為：張貼時間(時間戳)、張貼時間、版別、標題、撰文者、內文、標籤、網址、頭條標記</strong></span><strong>。資料格式為csv檔，並以UTF8編碼，直接存入Google雲端硬碟</strong></p>

<p>支援的分類新聞目錄：</p>

<p>1.台股 V<br />
&nbsp; a.台灣政經 V<br />
&nbsp; b.台股盤勢 V<br />
&nbsp; c.台股盤勢 V<br />
&nbsp; d.台股盤勢 V</p>

<p>2.國際股 V<br />
&nbsp; a.美股 V<br />
&nbsp; b.美股雷達 V<br />
&nbsp; c.歐亞股 V<br />
&nbsp; d.國際政經 V</p>

<p>3.陸港股 V</p>

<p>4.外匯 V</p>

<p>5.期貨 V</p>

<p>6.房產 V</p>

<p>7.基金 V</p>

<p>8.區塊鏈 V</p>


# 新聞類別與查詢區間設定


In [1]:
#設定區域
#==============================================
#頭條設定 
# 1表示限定頭條； 0表示不限 
Headline = 1

#輸入類別
c = '台股'

#輸入欲查詢之股票代號
code = '2330'

#查詢區間
s_time = "2022-01-01 00:00:00" #開始日期 
e_time = "2022-01-31 00:00:00" #結束日期

#檔名設定
file_name = '2330 original_data'

#==============================================


kind = {
  '台股':'tw_stock',
  '台灣政經':'tw_macro',
  '台股盤勢':'tw_quo',
  '專家觀點':'stock_report',
  '台股新聞':'tw_stock_news',

  '國際股':'wd_stock',
  '美股':'us_stock',
  '美股雷達':'us_stock_live',
  '歐亞股':'eu_asia_stock',
  '國際政經':'wd_macro',

  '陸港股':'cn_stock',
  '外匯':'forex',
  '期貨':'future',
  '房產':'cnyeshouse',
  '基金':'fund',
  '區塊鏈':'bc'

}

c = kind.get(c)


#頭條篩選
if Headline == 1 :
  CH = '&isCategoryHeadline=1'
else :
  CH = ''

# 安裝並載入套件

In [2]:
!pip install fake_useragent
from fake_useragent import UserAgent
import requests
from lxml import etree

import random
import time
import datetime
import json
import pandas as pd
#%reset -f

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=b83f188a6a09460d39b2bab3b5c4f10cad32ed0222209b6d4532340568052218
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


# 自定義程式：爬取鉅亨網新聞網址清單

In [6]:
def url_list(s_time,e_time):
  
  #==============================================
  #時間區間設定
  s_time = int(time.mktime(time.strptime(s_time, "%Y-%m-%d %H:%M:%S")))-28800 # 轉成時間戳
  e_time = int(time.mktime(time.strptime(e_time, "%Y-%m-%d %H:%M:%S")))-28800

  #==============================================
  p = 1
  last_page = 3
  data = pd.DataFrame()

  while p <= last_page :

    headers = UserAgent().random #隨機請求表頭
    
    url = 'https://api.cnyes.com/media/api/v1/newslist/category/'+c+'?startAt='+str(s_time)+'&endAt='+str(e_time)+CH+"&limit=30&page="+str(p)
    req = requests.get(url, headers={"User-Agent":headers})
    req = req.json()

    #最末頁之路徑
    if p == 1:
      last_page = req["items"]["last_page"] 

    req_data = req["items"]["data"]
    temp_data = pd.DataFrame.from_dict(req_data)
    
    #擷取資料
    publishAt = temp_data['publishAt']
    newsId = temp_data['newsId']
    category = temp_data['categoryName']
    title = temp_data['title']
    stock = temp_data['stock']
    isCategoryHeadline = temp_data['isCategoryHeadline']
    
    temp_data = pd.concat([publishAt, isCategoryHeadline, category, title, stock, newsId], axis=1)
    temp_data = reFormat(temp_data)
    
    data = pd.concat([data, temp_data], ignore_index=True)
    
    

    #休眠
    t = random.uniform(0.005, 0.1) #隨機數
    time.sleep(t) #暫停

    p = p+1

    if p%20 == 0:
      print(p/last_page*100)
      data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
      t = random.uniform(0.5, 1) #隨機數
      time.sleep(t)

  data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料

  
  return(data)



In [7]:
def reFormat (temp_data):
  for r in range(len(temp_data)):

    #擷取股票代號
    temp_data['stock'][r] = ','.join(temp_data['stock'][r])
    if len(temp_data['stock'][r]) == 0 :
      temp_data['stock'][r] = '0'


  return(temp_data)

# 執行程式

In [8]:
data_link = url_list(s_time,e_time)
data_link = data_link.sort_values(by='publishAt', ascending=True)
mark = data_link['stock'].str.contains(code)
data_link = data_link.loc[mark]
data_link = data_link.reset_index(drop=True) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


43.47826086956522
86.95652173913044


# 檢視新聞網址資料

In [9]:
data_link

,publishAt,isCategoryHeadline,categoryName,title,stock,newsId
0,1641031220,1,鉅亨新視界,〈觀察〉專業分工為王道 為何碳化矽產業是IDM廠的天下？,"2330,5483,6488,8086,8255",4795133
1,1641100483,1,台股新聞,〈台股風向球〉台積電領軍半導體鳴槍衝 高點不預設立場,2330,4795334
2,1641171916,1,台股盤勢,【Wen姐筆記】2022年開紅盤 短多控盤別掉球 外資歸隊增量指路,2330,4795523
3,1641175703,1,專家觀點,力智vs杰力比價 帶動MOSFET春天,"2330,2303,2454,6792,6121,6781,3592,2357,6719,2...",4795547
4,1641180441,1,台股新聞,台股基金去年平均報酬逾4成 兩優勢支撐元月行情火旺,2330,4795587
...,...,...,...,...,...,...
195,1643168679,1,台股新聞,〈新纖強攻半導體〉新碩先進製程用化學品明年Q1投產 就近供貨晶圓代工大廠,"1409,2330",4808350
196,1643182486,1,台股盤勢,〈台股盤後〉美股拖累 金牛年封關失守季線 全年上漲1872點,"2330,2303,2454,2317,3008,2609,2603,2615,2610,2...",4808428
197,1643187640,1,台股新聞,聯發科發表6奈米迅鯤Kompanio 1380 搶旗艦Chromebook商機,"2454,2330,2353,US-TSM,US-GOOG",4808550
198,1643194892,1,台股新聞,內外資不願抱股過年賣超178億元 聯電遭砍逾13萬張,"2303,2317,2330,3481,2337,2891,4906,1477,8046,3...",4808613


# 爬取新聞內文

In [10]:
data = pd.DataFrame()
l_page = len(data_link)

for i in range(l_page):

  headers = UserAgent().random
  url = data_link['newsId'][i]
  url = 'https://news.cnyes.com/news/id/'+str(url)

  req = requests.get(url, headers={"User-Agent":headers})
  req = req.content.decode()
  req = etree.HTML(req)

  publishAt = data_link['publishAt'][i]

  d_time = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/div[2]/time/text()')
  d_time = "".join(d_time)

  isCategoryHeadline = data_link['isCategoryHeadline'][i]

  catalog = data_link['categoryName'][i]

  title = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/h1/text()')
  title = "".join(title)

  reporter = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[2]/div[2]/span/span/text()')
  reporter = "".join(reporter)

  content = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/text() | //*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/div[2]/div/p/a/text()')
  content = "".join(content)
  if len(content) == 0:
    content = '0'

  stock = data_link['stock'][i]

  tag = req.xpath('//*[@id="content"]/div/div/div[2]/main/div[3]/article/section[1]/nav/a/span/text()')
  tag = ",".join(tag)


  temp_data = {(publishAt , d_time, isCategoryHeadline, catalog, title, reporter, content, stock, tag, url)}
  temp_data = pd.DataFrame(temp_data)
  data = data.append(temp_data)

  t = random.uniform(0.005, 0.1) #隨機數
  time.sleep(t) #暫停

  #列印出進度
  if i % 25 == 0:
    print(i/l_page*100)
    t = random.uniform(0.2, 1) #隨機數
    time.sleep(t)




0.0
12.5
25.0
37.5
50.0
62.5
75.0
87.5


# 檢視新聞內文資料

In [11]:
data.columns = [ 'publishAt', 'time', 'Headline', 'category', 'title', 'reporter', 'content', 'stock', 'tag', 'url']
data

,publishAt,time,Headline,category,title,reporter,content,stock,tag,url
0,1641031220,2022/01/01 18:00,1,鉅亨新視界,〈觀察〉專業分工為王道 為何碳化矽產業是IDM廠的天下？,鉅亨網記者林薏茹 台北,矽基半導體產業已高度垂直分工，台積電 (2330-TW) 更以專業晶圓代工的商業模式，站穩產...,"2330,5483,6488,8086,8255","化合物半導體,半導體,晶圓代工,SiC,基板,IDM",https://news.cnyes.com/news/id/4795133
0,1641100483,2022/01/02 13:14,1,台股新聞,〈台股風向球〉台積電領軍半導體鳴槍衝 高點不預設立場,鉅亨網記者王莞甯 台北,台股 2021 年封關前續漲 257.2 點，周線連 2 紅，本周將迎來 2022 年開紅盤...,2330,"台積電,台股,風向球,半導體,開紅盤",https://news.cnyes.com/news/id/4795334
0,1641171916,2022/01/03 09:05,1,台股盤勢,【Wen姐筆記】2022年開紅盤 短多控盤別掉球 外資歸隊增量指路,Wen姐,台股今天開出 2022 年紅盤，多頭是否能挾去年擺尾氣勢，繼續為今年搏得好彩頭? 首先今天耶...,2330,"台股,盤前,盤勢",https://news.cnyes.com/news/id/4795523
0,1641175703,2022/01/03 10:08,1,專家觀點,力智vs杰力比價 帶動MOSFET春天,楊惠宇(永誠國際投顧),2021 年，由疫情及寛鬆貨幣政策所帶動的通膨貫穿全年，雖然不平靜，但歐美股市都創下高點，亞...,"2330,2303,2454,6792,6121,6781,3592,2357,6719,2...","楊惠宇,永誠投顧,台股",https://news.cnyes.com/news/id/4795547
0,1641180441,2022/01/03 11:27,1,台股新聞,台股基金去年平均報酬逾4成 兩優勢支撐元月行情火旺,鉅亨網記者陳蕙綾 台北,外資在去 (2021) 年末大舉回流台股，帶動台股多頭氣焰猛，台股基金年度績效表現更直接受惠...,2330,"台股,台股基金,半導體,台積電,綠能,電動車",https://news.cnyes.com/news/id/4795587
...,...,...,...,...,...,...,...,...,...,...
0,1643168679,2022/01/26 11:44,1,台股新聞,〈新纖強攻半導體〉新碩先進製程用化學品明年Q1投產 就近供貨晶圓代工大廠,鉅亨網記者彭昱文 台南,新纖 (1409-TW) 強攻半導體精密化學品領域，與比利時化學大廠 Solvay 合資的子...,"1409,2330","新纖,半導體,先進製程,台積電",https://news.cnyes.com/news/id/4808350
0,1643182486,2022/01/26 15:34,1,台股盤勢,〈台股盤後〉美股拖累 金牛年封關失守季線 全年上漲1872點,鉅亨網記者郭幸宜 台北,美股重挫、俄烏緊繃情勢，台股金牛年封關日小跌收黑，終場下跌 26 點收在 17674 點，成...,"2330,2303,2454,2317,3008,2609,2603,2615,2610,2...","台股,封關,牛年,國安基金,升息,聯準會",https://news.cnyes.com/news/id/4808428
0,1643187640,2022/01/26 17:00,1,台股新聞,聯發科發表6奈米迅鯤Kompanio 1380 搶旗艦Chromebook商機,鉅亨網記者魏志豪 台北,聯發科 (2454-TW) 今 (26) 日發表迅鯤 Kompanio 1380 晶片，以台...,"2454,2330,2353,US-TSM,US-GOOG","聯發科,GOOGLE,Chromebook,6奈米,台積電",https://news.cnyes.com/news/id/4808550
0,1643194892,2022/01/26 19:01,1,台股新聞,內外資不願抱股過年賣超178億元 聯電遭砍逾13萬張,鉅亨網記者郭幸宜 台北,台股牛年封關但內外資抱股過年意願不高，三大法人合計賣超 178.46 億元，其中晶圓代工廠聯...,"2303,2317,2330,3481,2337,2891,4906,1477,8046,3...","台股,封關,聯電,牛年",https://news.cnyes.com/news/id/4808613


# 連結Google Drive並匯出資料

In [12]:
from google.colab import drive
drive.mount('/content/drive')


#輸出檔案
path = '/content/drive/MyDrive/DATA set/'+file_name+'_full.csv'
data.to_csv(path, index=None)

Mounted at /content/drive
